In [ ]:
!pip install -q --upgrade gspread
!pip install -q pandas

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
import re

Access to Hubspot Import

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
ss = gc.open_by_key('KEY')
hubspot_data = ss.worksheet('SHEETS')
hubsrows = hubspot_data.get_all_values()
df_hubs = pd.DataFrame.from_records(hubsrows[1:],columns=hubsrows[0])
display(df_hubs.columns)

In [ ]:
df_hubs

,Deal ID,Deal Name,MQL Date,SQL Date,SQL Value,Opportunity Date,Close Date,Amount in company currency,Deal Type,Industry,Head Industry
0,6884406584,BR: Netzsch Pumps,2021-11-09,,6175,,2021-11-30 14:04,,,Machinery,
1,6881926026,BR: 4all,2021-11-09,,,,2021-11-30 12:20,,,Information Technology and Services,
2,6881585841,BR: OJI PAPÉIS ESPECIAIS,2021-11-09,2021-11-10,6175,,2021-11-30 12:18,6175,,Paper & Forest Products,
3,6880888967,BR: Shopee Limited,2021-11-09,,58560,,2021-11-30 11:51,,New Subscription,Information Technology and Services,
4,6849082125,BR: Granito Pagamentos,2021-11-05,,19520,,2021-11-30 12:46,,New Subscription,Financial Services,
...,...,...,...,...,...,...,...,...,...,...,...
411,3961432193,Anjo Tintas,2021-01-08,,,,2021-01-29 08:47,,,,
412,3949492675,BR: Aperam,2021-01-06,2021-01-07,6033,2021-02-09,2021-05-06 13:02,6350,New Subscription,Mining & Metals,Energy and Natural Resources
413,3943035194,BR: upLexis,2021-01-05,2021-01-08,7172,,2021-02-19 08:30,7172,,Information Technology and Services,Technology & Design
414,3941682803,WS: flavio.conrado@sankhya.com.br,,,,,2021-01-11 13:56,,,,


##Clustering information needed from Hubspot

In [ ]:
df_hubs['MQL Date'] = pd.to_datetime(df_hubs['MQL Date'])
df_hubs['SQL Date'] = pd.to_datetime(df_hubs['SQL Date'])
df_hubs['Opportunity Date'] = pd.to_datetime(df_hubs['Opportunity Date'])

MQLs

In [ ]:
mql_count = df_hubs[['Deal ID','MQL Date']].groupby(pd.Grouper(key='MQL Date',freq='M')).count()
mql_count

,Deal ID
MQL Date,
2021-01-31,22
2021-02-28,18
2021-03-31,30
2021-04-30,38
2021-05-31,49
2021-06-30,45
2021-07-31,36
2021-08-31,56
2021-09-30,57


SQLs

In [ ]:
sql_count = df_hubs[['Deal ID','SQL Date']].groupby(pd.Grouper(key='SQL Date',freq='M')).count()
sql_count

,Deal ID
SQL Date,
2021-01-31,9
2021-02-28,12
2021-03-31,16
2021-04-30,19
2021-05-31,23
2021-06-30,28
2021-07-31,23
2021-08-31,22
2021-09-30,27


Opps

In [ ]:
opp_count = df_hubs[['Deal ID','Opportunity Date']].groupby(pd.Grouper(key='Opportunity Date',freq='M')).count()
opp_count

,Deal ID
Opportunity Date,
2021-01-31,4
2021-02-28,3
2021-03-31,6
2021-04-30,12
2021-05-31,14
2021-06-30,11
2021-07-31,14
2021-08-31,14
2021-09-30,15


In [ ]:
print(df_hubs.dtypes)

Deal ID                               object
Deal Name                             object
MQL Date                      datetime64[ns]
SQL Date                      datetime64[ns]
SQL Value                             object
Opportunity Date              datetime64[ns]
Close Date                            object
Amount in company currency            object
Deal Type                             object
Industry                              object
Head Industry                         object
dtype: object


SQL Value

In [ ]:
df_hubs['SQL Value'] = pd.to_numeric(df_hubs['SQL Value'])
sql_sum = df_hubs[['SQL Date','SQL Value']].groupby(pd.Grouper(key='SQL Date',freq='M')).sum()
sql_sum

,SQL Value
SQL Date,
2021-01-31,75361.0
2021-02-28,104249.0
2021-03-31,168623.0
2021-04-30,217430.0
2021-05-31,201787.0
2021-06-30,383675.0
2021-07-31,342909.0
2021-08-31,262248.0
2021-09-30,318531.0


Opps Value

In [ ]:
df_hubs['Amount in company currency'] = pd.to_numeric(df_hubs['Amount in company currency'])
opp_sum = df_hubs[['Opportunity Date','Amount in company currency']].groupby(pd.Grouper(key='Opportunity Date',freq='M')).sum()
opp_sum

,Amount in company currency
Opportunity Date,
2021-01-31,80150.0
2021-02-28,22052.0
2021-03-31,48337.0
2021-04-30,142187.0
2021-05-31,194653.0
2021-06-30,186680.0
2021-07-31,173059.0
2021-08-31,187160.0
2021-09-30,206109.0


Head and Sub Industries Dictionary

In [ ]:
ind_accesss = gc.open_by_key('1ASfhwu4pKwET5bz8coI59Bzpn05OuGoZBmWr12nQsyc')
ind_data = ind_accesss.worksheet('Industries')
indsrows = ind_data.get_all_values()
df_industries = pd.DataFrame.from_records(indsrows[1:],columns=indsrows[0])
display(df_industries.columns)
ind_dict = dict(zip(df_industries['Industry'],df_industries['Head Industry']))

Index(['#', 'Industry', 'Head Industry', '', '', 'HEAD INDUSTRIEs', '', '', '',
       ''],
      dtype='object')

In [ ]:
df_hubs['Head Industry New'] = df_hubs["Industry"].map(ind_dict)

MQL, SQL, OPP, Close by Head Industry

In [ ]:
ind_count = df_hubs[['Head Industry New','MQL Date', 'SQL Date', 'Opportunity Date','Close Date']].groupby(pd.Grouper(key='Head Industry New')).count()
ind_count

,MQL Date,SQL Date,Opportunity Date,Close Date
Head Industry New,,,,
Construction and Engineering,21,7,4,21
Technology & Design,51,27,18,53
Industrial Manufacturing,17,9,3,17
Financial Services,39,17,6,39
Professional Services,45,26,20,45
Consumer Goods,21,8,6,21
Food and Beverage,15,8,4,15
Hospitality and Tourism,9,4,2,9
Wholesale and Retail,9,5,3,9


MQL, SQL, OPP, Close Conversion by Industry

In [ ]:
ind_count['MQL to SQL'] = ind_count['SQL Date']/ind_count['MQL Date']
ind_count['SQL to OPP'] = ind_count['Opportunity Date']/ind_count['SQL Date']
ind_count_sort = ind_count.sort_index()
ind_count_sort

,MQL Date,SQL Date,Opportunity Date,Close Date,MQL to SQL,SQL to OPP
Head Industry New,,,,,,
"Agriculture, Forestry and Fishing",21,11,4,21,0.523810,0.363636
"Arts, Media and Entertainment",6,1,0,6,0.166667,0.000000
Automotive,23,13,7,24,0.565217,0.538462
Communications,11,2,1,12,0.181818,0.500000
Construction and Engineering,21,7,4,21,0.333333,0.571429
Consumer Goods,21,8,6,21,0.380952,0.750000
Education and Research,7,7,3,7,1.000000,0.428571
Energy and Natural Resources,46,29,15,46,0.630435,0.517241
Financial Services,39,17,6,39,0.435897,0.352941


## Clustering information needed from Prospection

In [ ]:
p3 = gc.open_by_key('12GRG8PNzKhkhVcuUcrLWXvVLxz58gdF_0zVy4vS24NU')
p3_data = p3.worksheet('All')
p3rows = p3_data.get_all_values()
df_p3 = pd.DataFrame.from_records(p3rows[1:],columns=p3rows[0])
display(df_p3.columns)

Index(['Date', 'Company Name', 'Company ID', 'S-ID', 'LGA', 'Country', 'City',
       'Score Corp', 'Course 1', 'Course 2', 'website', 'First Name',
       'Last Name', 'Name', 'Score', 'Title', 'Email', 'LinkedIn Profile',
       'Phone', 'Industry', 'NeverBounce'],
      dtype='object')

In [ ]:
p2 = gc.open_by_key('1XGrArQUq3O1uo7oYRPeKtPczGBkxd4v-yJtBdy_SZmQ')
p2_data = p2.worksheet('Sheet1')
p2rows = p2_data.get_all_values()
df_p2 = pd.DataFrame.from_records(p2rows[1:],columns=p2rows[0])
display(df_p2.columns)

Index(['Date', 'Company Name', 'Company ID', 'S-ID', 'LGA', 'Country', 'City',
       'Score Corp', 'Course 1', 'Course 2', 'website', 'First Name',
       'Last Name', 'Name', 'Score', 'Title', 'Email', 'LinkedIn Profile',
       'Phone', 'Industry', 'NeverBounce'],
      dtype='object')

Merge Prospects 2 and 3

In [ ]:
prospects_df = pd.concat([df_p2,df_p3],axis=0)

In [ ]:
prospects_df['Country'].unique()

array(['Colombia', 'Mexico', 'Chile', 'Argentina', 'Peru', 'Costa Rica',
       'Ecuador', 'Guatemala', 'España', 'Honduras', 'Panama', 'Uruguay',
       'Republica Dominicana', 'Paraguay', 'Bolivia', 'El Salvador',
       'Puerto Rico', 'Brasil', 'Nicaragua', 'United States',
       'Estados Unidos', 'Costa RIca', 'USA', 'Australia', 'México', ''],
      dtype=object)

Brasil's data

In [ ]:
prospects_br_df = prospects_df[(prospects_df["Country"].isin(['Brasil']))] 
prospects_br_df.columns

Index(['Date', 'Company Name', 'Company ID', 'S-ID', 'LGA', 'Country', 'City',
       'Score Corp', 'Course 1', 'Course 2', 'website', 'First Name',
       'Last Name', 'Name', 'Score', 'Title', 'Email', 'LinkedIn Profile',
       'Phone', 'Industry', 'NeverBounce'],
      dtype='object')

Leads by Month

In [ ]:
prospects_br_df['Date'] = pd.to_datetime(prospects_br_df['Date'])
prospects_br_df2021 = prospects_br_df[prospects_br_df['Date'] > '2020-12-31']
prosp_companies_br = prospects_br_df2021[['Date','Company Name']].groupby(pd.Grouper(key='Date',freq='M')).count()
prosp_companies_br

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Company Name
Date,
2021-01-31,1141
2021-02-28,935
2021-03-31,1149
2021-04-30,1522
2021-05-31,1747
2021-06-30,1582
2021-07-31,1773
2021-08-31,2725
2021-09-30,1896


Leads by Head Industry

In [ ]:
prospects_br_df2021['Head Industry New'] = prospects_br_df2021["Industry"].map(ind_dict)
prosp_ind_br = prospects_br_df2021[['Head Industry New','Company Name']].groupby(pd.Grouper(key='Head Industry New')).count()
prosp_ind_br_sort = prosp_ind_br.sort_index()
prosp_ind_br_sort

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Company Name
Head Industry New,
"Agriculture, Forestry and Fishing",821
"Arts, Media and Entertainment",257
Automotive,982
Communications,433
Construction and Engineering,928
Consumer Goods,775
Education and Research,33
Energy and Natural Resources,1763
Financial Services,1687


## Joining Tables

Joining Hubspot's Metrics

In [ ]:
part1 = pd.merge(mql_count, sql_count, left_index=True, right_index=True)
part2 = pd.merge(part1,opp_count,left_index=True,right_index=True)
part3 = pd.merge(part2,sql_sum,left_index=True,right_index=True)
part4 = pd.merge(part3,opp_sum,left_index=True,right_index=True)
part4
part4r = part4.rename(columns={'Deal ID_x':'MQLs','Deal ID_y':'SQLs','Deal ID':'Opps','Amount in company currency':'Opp Value'})

Joining Hubspot and Prospects

In [ ]:
part5 = pd.merge(part4r,prosp_companies_br,left_index=True,right_index=True)
Hubs_merge = part5.rename(columns={'Company Name':'Leads'})
Hubs_merge['Lead-SQL'] = Hubs_merge['SQLs']/Hubs_merge['Leads']
Hubs_merge['Month'] = Hubs_merge.index
Hubs_merge

,MQLs,SQLs,Opps,SQL Value,Opp Value,Leads,Lead-SQL,Month
MQL Date,,,,,,,,
2021-01-31,22,9,4,75361.0,80150.0,1141,0.007888,2021-01-31
2021-02-28,18,12,3,104249.0,22052.0,935,0.012834,2021-02-28
2021-03-31,30,16,6,168623.0,48337.0,1149,0.013925,2021-03-31
2021-04-30,38,19,12,217430.0,142187.0,1522,0.012484,2021-04-30
2021-05-31,49,23,14,201787.0,194653.0,1747,0.013165,2021-05-31
2021-06-30,45,28,11,383675.0,186680.0,1582,0.017699,2021-06-30
2021-07-31,36,23,14,342909.0,173059.0,1773,0.012972,2021-07-31
2021-08-31,56,22,14,262248.0,187160.0,2725,0.008073,2021-08-31
2021-09-30,57,27,15,318531.0,206109.0,1896,0.014241,2021-09-30


Industries from Hubspot and Prospects

In [ ]:
part6 = pd.merge(ind_count,prosp_ind_br,left_index=True,right_index=True)
Ind_merge = part6.rename(columns={'Company Name':'Leads','MQL Date':'MQLs','SQL Date':'SQLs','Opportunity Date':'Opps'})
Ind_merge['Lead to MQL'] = Ind_merge['MQLs']/Ind_merge['Leads']
Ind_merge['Industries'] = Ind_merge.index
Ind_merge

,MQLs,SQLs,Opps,Close Date,MQL to SQL,SQL to OPP,Leads,Lead to MQL,Industries
Head Industry New,,,,,,,,,
Construction and Engineering,21,7,4,21,0.333333,0.571429,928,0.022629,Construction and Engineering
Technology & Design,51,27,18,53,0.529412,0.666667,1879,0.027142,Technology & Design
Industrial Manufacturing,17,9,3,17,0.529412,0.333333,1030,0.016505,Industrial Manufacturing
Financial Services,39,17,6,39,0.435897,0.352941,1687,0.023118,Financial Services
Professional Services,45,26,20,45,0.577778,0.769231,1404,0.032051,Professional Services
Consumer Goods,21,8,6,21,0.380952,0.750000,775,0.027097,Consumer Goods
Food and Beverage,15,8,4,15,0.533333,0.500000,1001,0.014985,Food and Beverage
Hospitality and Tourism,9,4,2,9,0.444444,0.500000,280,0.032143,Hospitality and Tourism
Wholesale and Retail,9,5,3,9,0.555556,0.600000,696,0.012931,Wholesale and Retail


## Update to Spreadsheets

In [ ]:
wsUpdate = ss.worksheet("Py Update")

In [ ]:
Hubs_merge.fillna('', inplace=True)
Hubs_merge['Month'] = Hubs_merge['Month'].astype(str)
aoa_metrics = [Hubs_merge.columns.tolist()] + Hubs_merge.to_numpy().tolist()
wsUpdate.update("A1",aoa_metrics)

{'spreadsheetId': '1uBZT-fPrMK3mY2SnzA6SZ_yGZjHmc7oWjE4tYBG4AAU',
 'updatedCells': 96,
 'updatedColumns': 8,
 'updatedRange': "'Py Update'!A1:H12",
 'updatedRows': 12}

In [ ]:
Ind_merge.fillna('', inplace=True)
aoa_ind = [Ind_merge.columns.tolist()] + Ind_merge.to_numpy().tolist()
wsUpdate.update("J1",aoa_ind)